<a href="https://colab.research.google.com/github/aryan-at-ul/image_segmentation/blob/main/pneumonia_image_as_graph_classification_resnet_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install imageio==2.22.4
!pip install llvmlite==0.39.1
!pip install matplotlib==3.6.2
!pip install networkx==2.8.8
!pip install numba==0.56.4
!pip install numpy
!pip install opencv-python==4.6.0.66
!pip install psutil==5.9.4
!pip install pytz==2022.6
!pip install scikit-image==0.19.3
!pip install scipy 
!pip install timm==0.6.12
!pip install torch==1.13.0
!pip install torchinfo==1.7.1
!pip install torchvision==0.14.0
# tqdm @ file:///opt/conda/conda-bld/tqdm_1647339053476/work
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 
!pip install gdown

In [2]:
from torch_geometric.data import Data
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
from torch_geometric.loader import DataLoader
import os
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, JumpingKnowledge
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import numpy as np
from torch_geometric.nn import MLP, DynamicEdgeConv, global_max_pool
import torch.nn.functional as F
import pickle
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
# from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GCNConv
import random
from torch.nn import Linear, ReLU, Dropout
from torch_geometric.nn import Sequential, GCNConv, JumpingKnowledge
from torch_geometric.nn import global_mean_pool
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

In [ ]:
!curl -o train_dataloader.pkl -L 'https://drive.google.com/uc?export=download&confirm=yes&id=1luFl1j_zf07eD_CHrpprXPz1udCIhjUU'
!curl -o test_dataloader.pkl -L 'https://drive.google.com/uc?export=download&confirm=yes&id=115kJ5EzpCL7TXLKUotJXuK6ngwcyC84E'
!curl -o val_dataloader.pkl -L 'https://drive.google.com/uc?export=download&confirm=yes&id=1SlQqiqT6vAlotmT9xc5PMQdeFLTh_PUa'

In [ ]:
!du -sh * 

In [26]:

curretn_path = os.getcwd()
path = f"{curretn_path}"

embed_dim = 128

def load_all_from_one_folder(path,type = 0):
    all_files = os.listdir(path)
    all_data = []
    k = 0
    for one_g in all_files:
        name = one_g.split(".")[0]
        G = nx.read_gpickle(f"{path}/{one_g}")  

        data = from_networkx(G)
        if type:
            data.y = [1]
        else:
            data.y = [0]
        k+= 1
        data.x = torch.Tensor([torch.flatten(val).tolist() for val in data.x])#nx.get_node_attributes(G,'image')
        data.name = name
        all_data.append(data)
    return all_data


def permute_array(array):
    permuted_array = []
    for i in range(len(array)):
        permuted_array.append(array[i])
    return permuted_array

def check_if_a_with_name_exisi(path,name):
    all_files = os.listdir(path)
    if name in all_files:
        return True
    else:
        return False





def dataloader():
    """
    load train and test data
    """
    print("loading data")
    train_dataset, test_dataset, val_dataset = None, None, None

    if not check_if_a_with_name_exisi(curretn_path,'train_dataloader.pkl'):

        train_normal = load_all_from_one_folder(f"{path}/train/NORMAL")
        train_pneumonia = load_all_from_one_folder(f"{path}/train/PNEUMONIA",1)

        test_normal = load_all_from_one_folder(f"{path}/test/NORMAL")
        test_pneumonia = load_all_from_one_folder(f"{path}/test/PNEUMONIA",1)

        val_normal = load_all_from_one_folder(f"{path}/val/NORMAL")
        val_pneumonia = load_all_from_one_folder(f"{path}/val/PNEUMONIA",1)


        train_data_arr = train_normal + train_pneumonia
        test_data_arr = test_normal + test_pneumonia
        val_data_arr = val_normal + val_pneumonia
        # all_data = permute_array(all_data)
        random.shuffle(train_data_arr)
        random.shuffle(test_data_arr)
        random.shuffle(val_data_arr)
        
        train_dataset = train_data_arr#all_data[:int(len(all_data)*0.8)]
        val_dataset = val_data_arr#all_data[int(len(all_data)*0.8):int(len(all_data)*0.8) + 100]
        test_dataset = test_data_arr#all_data[int(len(all_data)*0.8):]
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False,drop_last=True)
        # test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True,drop_last=True)
        test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True,drop_last=True)

        with open('train_dataloader.pkl', 'wb') as f:
            pickle.dump(train_loader, f)

        with open('test_dataloader.pkl', 'wb') as f:
            pickle.dump(test_loader, f)
        
        with open('val_dataloader.pkl', 'wb') as f:
            pickle.dump(val_loader, f)
    else:
        with open('train_dataloader.pkl', 'rb') as f:
            train_loader = pickle.load(f)

        with open('test_dataloader.pkl', 'rb') as f:
            test_loader = pickle.load(f)
        
        with open('val_dataloader.pkl', 'rb') as f:
            val_loader = pickle.load(f)


    return train_loader, test_loader, train_dataset, test_dataset, val_loader, val_dataset



train_loader, test_loader, train_dataset, test_dataset, val_loader, val_dataset = dataloader()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("data loaded")

loading data
data loaded


In [27]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(512, 256)
        self.conv2 = GCNConv(256, 128)
        self.conv3 = GCNConv(128,64)
        self.conv4 = GCNConv(64, 32)
        self.lin1 = Linear(32, 16)
        self.lin = Linear(16, 2)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)

        x = global_mean_pool(x, batch)  

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=512) # based on feature size

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.001)
criterion = torch.nn.CrossEntropyLoss()

In [29]:
def train():
    model.train()

    for data in train_loader: 
        out = model(data.x, data.edge_index, data.batch) 
        data.y = torch.Tensor(data.y)
        data.y = torch.Tensor(torch.flatten(data.y))
        data.y = data.y.type(torch.LongTensor)
        loss = criterion(out, data.y)
        loss.backward()  
        optimizer.step()
        optimizer.zero_grad()  

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  
         out = model(data.x, data.edge_index, data.batch)  
         data.y = torch.Tensor(data.y)
         pred = out.argmax(dim=1).view(-1,1)  
         correct += int((pred == data.y).sum())  
         acc = correct / len(loader.dataset)
        #  if acc > 0.91:
        #      torch.save(model.state_dict(), 'model_res_10sp.pt')
     return correct / len(loader.dataset) 





In [ ]:
for epoch in range(1, 21):
    train()
    try:
        train_acc = test(train_loader)
        test_acc = test(test_loader)
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    except Exception as e:
        print("error",e)
        pass

print("number of paramteres for this model",sum(p.numel() for p in model.parameters()))

Epoch: 001, Train Acc: 0.9699, Test Acc: 0.7933
Epoch: 002, Train Acc: 0.9787, Test Acc: 0.8478
Epoch: 003, Train Acc: 0.9799, Test Acc: 0.8510
Epoch: 004, Train Acc: 0.9804, Test Acc: 0.8381
Epoch: 005, Train Acc: 0.9795, Test Acc: 0.8413
Epoch: 006, Train Acc: 0.9795, Test Acc: 0.8157
Epoch: 007, Train Acc: 0.9814, Test Acc: 0.8237
Epoch: 008, Train Acc: 0.9814, Test Acc: 0.8478
Epoch: 009, Train Acc: 0.9793, Test Acc: 0.8478
Epoch: 010, Train Acc: 0.9711, Test Acc: 0.7724
